In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 
import catboost as cb#catboostregressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import mean_squared_error, confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/coffee-quality-data-cqi/df_arabica_clean.csv


In [25]:
data = pd.read_csv("/kaggle/input/coffee-quality-data-cqi/df_arabica_clean.csv")
data.head()

,Unnamed: 0,ID,Country of Origin,Farm Name,Lot Number,Mill,ICO Number,Company,Altitude,Region,...,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Color,Category Two Defects,Expiration,Certification Body,Certification Address,Certification Contact
0,0,0,Colombia,Finca El Paraiso,CQU2022015,Finca El Paraiso,NaN,Coffee Quality Union,1700-1930,"Piendamo,Cauca",...,89.33,11.8,0,0,green,3,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
1,1,1,Taiwan,Royal Bean Geisha Estate,"The 2022 Pacific Rim Coffee Summit,T037",Royal Bean Geisha Estate,NaN,Taiwan Coffee Laboratory,1200,Chiayi,...,87.58,10.5,0,0,blue-green,0,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
2,2,2,Laos,OKLAO coffee farms,"The 2022 Pacific Rim Coffee Summit,LA01",oklao coffee processing plant,NaN,Taiwan Coffee Laboratory,1300,Laos Borofen Plateau,...,87.42,10.4,0,0,yellowish,2,"November 15th, 2023",Taiwan Coffee Laboratory 台灣咖啡研究室,"QAHWAH CO., LTD 4F, No. 225, Sec. 3, Beixin Rd...","Lin, Jen-An Neil 林仁安 - 886-289116612"
3,3,3,Costa Rica,La Cumbre,CQU2022017,La Montana Tarrazu MIll,NaN,Coffee Quality Union,1900,"Los Santos,Tarrazu",...,87.17,11.8,0,0,green,0,"September 21st, 2023",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901
4,4,4,Colombia,Finca Santuario,CQU2023002,Finca Santuario,NaN,Coffee Quality Union,1850-2100,"Popayan,Cauca",...,87.08,11.6,0,2,yellow-green,2,"March 5th, 2024",Japan Coffee Exchange,"〒413-0002 静岡県熱海市伊豆山１１７３−５８ 1173-58 Izusan, Ata...",松澤 宏樹 Koju Matsuzawa - +81(0)9085642901


In [26]:
data['Harvest Year'] = data['Harvest Year'].str.split('/').str[0].str.strip()
data['Harvest Year'] = pd.to_datetime(data['Harvest Year'], format='%Y')
data['Expiration'] = data['Expiration'].str.split(',').str[-1].str.strip()
data['Expiration'] = pd.to_datetime(data['Expiration'], format='%Y')

In [27]:
data['Coffee Age'] = (data['Expiration'] - data['Harvest Year']).dt.days

In [28]:
data['Coffee Age']

0      730
1      730
2      730
3      365
4      730
      ... 
202    730
203    730
204    730
205    365
206    730
Name: Coffee Age, Length: 207, dtype: int64

In [29]:
data.columns

Index(['Unnamed: 0', 'ID', 'Country of Origin', 'Farm Name', 'Lot Number',
       'Mill', 'ICO Number', 'Company', 'Altitude', 'Region', 'Producer',
       'Number of Bags', 'Bag Weight', 'In-Country Partner', 'Harvest Year',
       'Grading Date', 'Owner', 'Variety', 'Status', 'Processing Method',
       'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance',
       'Uniformity', 'Clean Cup', 'Sweetness', 'Overall', 'Defects',
       'Total Cup Points', 'Moisture Percentage', 'Category One Defects',
       'Quakers', 'Color', 'Category Two Defects', 'Expiration',
       'Certification Body', 'Certification Address', 'Certification Contact',
       'Coffee Age'],
      dtype='object')

In [30]:
drop_col = ['ID','ICO Number','Owner','Region','Certification Contact','Certification Address','Farm Name',"Lot Number","Mill","ICO Number","Producer",'Company','Expiration', 'Harvest Year',
                   "Unnamed: 0",'Number of Bags','Bag Weight','In-Country Partner','Grading Date','Variety','Status','Defects','Uniformity','Clean Cup','Sweetness','Certification Body', 
                  'Country of Origin', 'Color', 'Altitude', 'Processing Method']
data.drop(drop_col, axis=1, inplace=True)

In [31]:
data

,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Overall,Total Cup Points,Moisture Percentage,Category One Defects,Quakers,Category Two Defects,Coffee Age
0,8.58,8.50,8.42,8.58,8.25,8.42,8.58,89.33,11.8,0,0,3,730
1,8.50,8.50,7.92,8.00,7.92,8.25,8.50,87.58,10.5,0,0,0,730
2,8.33,8.42,8.08,8.17,7.92,8.17,8.33,87.42,10.4,0,0,2,730
3,8.08,8.17,8.17,8.25,8.17,8.08,8.25,87.17,11.8,0,0,0,365
4,8.33,8.33,8.08,8.25,7.92,7.92,8.25,87.08,11.6,0,2,2,730
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,7.17,7.17,6.92,7.17,7.42,7.17,7.08,80.08,11.4,0,0,4,730
203,7.33,7.08,6.75,7.17,7.42,7.17,7.08,80.00,10.4,0,2,12,730
204,7.25,7.17,7.08,7.00,7.08,7.08,7.00,79.67,11.6,0,9,11,730
205,6.50,6.75,6.75,7.17,7.08,7.00,6.83,78.08,11.0,0,12,13,365


In [32]:
X = data.drop('Total Cup Points', axis = 1)
y = data['Total Cup Points']

In [33]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
model = RandomForestRegressor(n_estimators=100, criterion="friedman_mse")
model.fit(x_train, y_train)

RandomForestRegressor(criterion='friedman_mse')

In [34]:
y_pred = model.predict(x_test)
y_pred

array([82.3349, 86.0051, 84.5542, 83.8754, 81.8309, 86.2175, 83.721 ,
       83.3117, 85.4087, 82.508 , 82.0813, 85.3196, 82.5914, 79.7715,
       83.5843, 80.6003, 82.1326, 81.5477, 84.1107, 84.2035, 83.1514,
       82.2971, 83.7545, 83.3757, 84.407 , 85.4111, 83.9772, 85.8193,
       82.7173, 81.881 , 82.5152, 84.1463, 84.6546, 83.4088, 82.3202,
       83.9878, 82.9127, 82.2504, 85.0714, 83.5929, 84.6688, 81.9298])

In [35]:
mse = mean_squared_error(y_test, y_pred)
mse

0.10852291285714069

In [36]:
rmse = mse**.5
rmse

0.32942816038878747

In [40]:
cat_model = cb.CatBoostRegressor()
cat_model.fit(x_train, y_train)
cat_ypred = cat_model.predict(x_test)
cat_mse = mean_squared_error(y_test, cat_ypred)

Learning rate set to 0.030799
0:	learn: 1.7218486	total: 512us	remaining: 512ms
1:	learn: 1.6897684	total: 1.69ms	remaining: 842ms
2:	learn: 1.6536668	total: 2.14ms	remaining: 711ms
3:	learn: 1.6210112	total: 2.55ms	remaining: 635ms
4:	learn: 1.5885125	total: 2.96ms	remaining: 590ms
5:	learn: 1.5562309	total: 3.42ms	remaining: 566ms
6:	learn: 1.5250566	total: 3.79ms	remaining: 538ms
7:	learn: 1.4920138	total: 4.37ms	remaining: 542ms
8:	learn: 1.4623508	total: 5.26ms	remaining: 579ms
9:	learn: 1.4353448	total: 5.75ms	remaining: 569ms
10:	learn: 1.4067254	total: 6.11ms	remaining: 550ms
11:	learn: 1.3770428	total: 6.53ms	remaining: 538ms
12:	learn: 1.3522527	total: 7.52ms	remaining: 571ms
13:	learn: 1.3243285	total: 7.94ms	remaining: 560ms
14:	learn: 1.2974619	total: 8.29ms	remaining: 544ms
15:	learn: 1.2745388	total: 8.63ms	remaining: 531ms
16:	learn: 1.2493608	total: 8.99ms	remaining: 520ms
17:	learn: 1.2232500	total: 9.33ms	remaining: 509ms
18:	learn: 1.1985900	total: 9.68ms	remaining:

In [42]:
cat_rmse = (np.sqrt(mean_squared_error(y_test, cat_ypred)))
cat_rmse

0.30919262762105215